In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests, json
import os
# Custom imports
from src.utils import *

# Setting up the variables

In [ ]:
token = os.environ["NOTION_TOKEN"]
databaseId = os.environ["NOTION_CONTENT_BOARD_DB_ID"]
update_DB_URL = f'https://api.notion.com/v1/databases/{databaseId}'

headers = {
    "Authorization": "Bearer " + token,
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"}

## Get all the ideas from the Content Board

In [ ]:
db_data = readDatabase(databaseId, headers)

In [ ]:
db_data

In [ ]:
content_status_tags = ["Idea", "Drafts", "Next Improvements", "Edit", "Publish"]
for tag in content_status_tags:
    print(f"Tag: {tag} - ", get_idea_from_content_board(db_data, content_board_tag=tag))

## Updating a Content Status Tag

In [ ]:
page_title = "Article - 10 Tips for Better ML Dev"
property_name = "Status"
property_type = "select"
property_dest_name = "Drafts"
db_data = readDatabase(databaseId, headers)
page_content_idx = get_content_title_idx(db_data, page_title)
page_data = db_data["results"][page_content_idx]
pageId = page_data["id"]
update_PAGE_URL = f'https://api.notion.com/v1/pages/{pageId}'
print(f"Page ID: {pageId}")
print("Removing meta keys (last edit time, create by, etc...)")
page_data = remove_page_metadata(page_data, keys_to_remove=["last_edited_time","created_time","last_edited_by","created_by"])
page_data = remove_property_metadata(page_data, property_name, property_type, keys_to_remove=["id", "color"])
print(f"Updating the {property_name} option...")
update_page_property(page_data,pageId, headers, property_name, property_type, property_dest_name)

## Reading Metadata on Notion Page

In [ ]:
pageId = "88d4370d8f40431dbfa03c938788df80"
page_title = "Python Code"
page_data = readPage(pageId, headers)
print(f"Page ID: {pageId}")
print("Removing meta keys (last edit time, create by, etc...)")
page_data
# page_data = remove_page_metadata(page_data, keys_to_remove=["last_edited_time","created_time","last_edited_by","created_by"])
# page_data = remove_property_metadata(page_data, property_name, property_type, keys_to_remove=["id", "color"])
# print(f"Updating the {property_name} option...")
# update_page_property(page_data,pageId, headers, property_name, property_type, property_dest_name)

## Get Block Content (code)

In [ ]:
def readBlock(pageId, headers):
    readUrl = f"https://api.notion.com/v1/blocks/{pageId}/children"

    res = requests.request("GET", readUrl, headers=headers)
    data = res.json()
    print(res.status_code) 
    
    return data


#pageId = "88d4370d8f40431dbfa03c938788df80"
blockId = "88d4370d8f40431dbfa03c938788df80#ae37ddbbbbb842cd8010b4bdf2fd47be"
page_title = "Python Code"
block_data = readBlock(pageId, headers)
block_data

In [ ]:
block_data.keys()

In [ ]:
block_data["results"][0]["code"]["rich_text"][0]["plain_text"]

## Add Block Content

In [ ]:
# curl -X PATCH https://api.notion.com/v1/blocks/16d8004e-5f6a-42a6-9811-51c22ddada12/children \
#   -H 'Authorization: Bearer '"$NOTION_API_KEY"'' \
#   -H "Content-Type: application/json" \
#   -H "Notion-Version: 2022-06-28" \
#   --data '{
#     "children": [
#     {
#       "object": "block",
#       "type": "paragraph",
#       "paragraph": {
#         "text": [{ "type": "text", "text": { "content": "– Notion API Team", "link": { "type": "url", "url": "https://twitter.com/NotionAPI" } } }]
#       }
#     }
#   ]
# }'    

def add_content_to_page(pageId, block_content, headers):
      blockURL = f"https://api.notion.com/v1/blocks/{pageId}/children"
      res = requests.request("PATCH", blockURL, headers=headers, data=json.dumps(block_content))
      print(res.status_code)
      print(res.text)
      

In [ ]:
with open("./notion_py_cli.py", "r") as file:
    py_script_content = file.read()

pageId = ""
page_title = "Workflow Test Page"
page_data = readPage(pageId, headers)
page_data

In [ ]:
block_content = {
  "children": [
    {"type": "code",
      "code": {"language": "python","rich_text": 
        [{"plain_text": "", "text": {"content": f"{py_script_content}"}}]
    }
      }
  ]
}
add_content_to_page(pageId, block_content, headers)